# Predicciones Privadas con Syft Keras

## Paso 3: Solicitando Predicciones (Cliente)

Felicidades! Después de entrenar tu modelo con Keras y asegurarlo con Syft Keras, estás listo para solicitar predicciones privadas

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import syft as sy

## Datos

Aquí vamos a preprocesar nuestros datos de MNIST. Esto es idéntico a como preprocesamos durante el aprendizaje

In [ ]:
# Dimensiones de la imagen de entrada
img_rows, img_cols = 28, 28

# Divide los datos en aprendizaje y evaluación
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Conectando con el modelo

Antes de enviar solicitudes al modelo, necesitas conectarte a él. Para esto, puedes crear un cliente. Luego define los mismos 3 TFEWorkers (`alice`, `bob` y `carol`) y clúster. Finalmente, llama `connect_to_model`. Esto crea un servidor TFE de colas en el lado del cliente que conecta al servidor de solicitudes que ha sido instaurado por `model.serve()` en la **Parte 13b**. La cola será responsable de compartir el secreto de datos antes de entregar las acciones en una solicitud de predicción.

In [ ]:
num_classes = 10
input_shape = (1, 28, 28, 1)
output_shape = (1, num_classes)

In [ ]:
client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')
cluster = sy.TFECluster(alice, bob, carol)

client.connect_to_model(input_shape, output_shape, cluster)

## Consultar el modelo!

Todo listo para obtener predicciones privadas! Ejecutando `query_model` insertará la `image` en la cola creada arriba, compartir el secreto de datos localmente, y entregar las acciones al servidor del modelo de la **Parte 13b**.

In [ ]:
# Parámetros del usuario
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [ ]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "wrongly",
        predicted_label))

Esto es genial. Ahora eres capaz de clasificar estas tres imágenes correctamente! Pero lo que es especial sobre estas predicciones es que tú no has tenido que revelar ninguna información privada para obtener este servicio. El host del modelo nunca vio tus datos de entrada o tus predicciones, y tu nunca descargaste el modelo. Pudiste obtener las predicciones de manera privada, con datos encriptados y un modelo encriptado!

Antes de que nos apuremos a aplicar esto en nuestras apps, regresemos a la **Parte 13b** y limpiemos nuestro modelo!
